In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Ajouter le dossier parent au chemin pour importer les modules
sys.path.append('..')

# Importer les fonctions du module model_training
from src.model_training import (
    load_data,
    train_simple_linear_regression,
    train_multiple_linear_regression,
    visualize_predictions
)

# 1. Chargement des données
print("Chargement des données d'entraînement et de test...")
X_train, y_train, X_test, y_test = load_data(
    "../data/splits/train.csv", 
    "../data/splits/test.csv"
)

# Affichage des premières lignes des données d'entraînement
print("\nAperçu des données d'entraînement:")
X_train_head = X_train.head()
print(X_train_head)

# Statistiques descriptives des prix
print("\nStatistiques descriptives des prix (cible):")
print(y_train.describe())

# 2. Sélection d'une feature pour la régression simple
# Rechercher une feature numérique intéressante pour la régression simple
numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns

# Calculer la corrélation avec le prix pour choisir la meilleure feature
if len(numeric_features) > 0:
    correlations = []
    for feature in numeric_features:
        corr = y_train.corr(X_train[feature])
        correlations.append((feature, corr))
    
    # Trier par corrélation absolue décroissante
    correlations.sort(key=lambda x: abs(x[1]), reverse=True)
    
    print("\nCorrelations des features numériques avec le prix:")
    for feature, corr in correlations:
        print(f"{feature}: {corr:.4f}")
    
    # Choisir la feature avec la plus forte corrélation
    best_feature = correlations[0][0]
    print(f"\nFeature avec la plus forte corrélation: {best_feature} ({correlations[0][1]:.4f})")
else:
    # Si aucune feature numérique, utiliser la première colonne disponible
    best_feature = X_train.columns[0]
    print(f"\nAucune feature numérique trouvée, utilisation de: {best_feature}")

# 3. Régression linéaire simple
print("\n========================")
print("RÉGRESSION LINÉAIRE SIMPLE")
print("========================")

# Visualiser la relation entre la feature sélectionnée et le prix
plt.figure(figsize=(10, 6))
plt.scatter(X_train[best_feature], y_train, alpha=0.5)
plt.xlabel(best_feature)
plt.ylabel('Prix')
plt.title(f'Relation entre {best_feature} et prix')
plt.grid(True)
plt.show()

# Entraîner le modèle de régression simple
model_simple, metrics_simple, y_pred_simple = train_simple_linear_regression(
    X_train, X_test, y_train, y_test, best_feature
)

# Visualiser les prédictions
visualize_predictions(y_test, y_pred_simple, f"Régression Linéaire Simple ({best_feature})")
plt.show()

# 4. Régression linéaire multiple
print("\n========================")
print("RÉGRESSION LINÉAIRE MULTIPLE")
print("========================")

# Entraîner le modèle de régression multiple
model_multiple, metrics_multiple, y_pred_multiple = train_multiple_linear_regression(
    X_train, X_test, y_train, y_test
)

# Visualiser les prédictions
visualize_predictions(y_test, y_pred_multiple, "Régression Linéaire Multiple")
plt.show()

# 5. Comparaison des modèles
print("\n========================")
print("COMPARAISON DES MODÈLES")
print("========================")

comparison = pd.DataFrame({
    'Métrique': ['MSE (test)', 'RMSE (test)', 'R² (test)'],
    f'Régression Simple ({best_feature})': [
        metrics_simple['test_mse'],
        metrics_simple['test_rmse'],
        metrics_simple['test_r2']
    ],
    'Régression Multiple': [
        metrics_multiple['test_mse'],
        metrics_multiple['test_rmse'],
        metrics_multiple['test_r2']
    ]
})

print(comparison)

# Visualisation comparative
fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.35
index = np.arange(3)

bar1 = ax.bar(index, [
    metrics_simple['test_mse'], 
    metrics_simple['test_rmse'], 
    metrics_simple['test_r2']], 
    bar_width, label=f'Régression Simple ({best_feature})')

bar2 = ax.bar(index + bar_width, [
    metrics_multiple['test_mse'], 
    metrics_multiple['test_rmse'], 
    metrics_multiple['test_r2']], 
    bar_width, label='Régression Multiple')

ax.set_xlabel('Métriques')
ax.set_ylabel('Valeurs')
ax.set_title('Comparaison des performances des modèles')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(['MSE', 'RMSE', 'R²'])
ax.legend()
plt.tight_layout()

# Sauvegarder le graphique de comparaison
os.makedirs("../results", exist_ok=True)
plt.savefig("../results/model_comparison.png")
plt.show()


# 6. ANALYSE APPROFONDIE ET MODÈLE AVANCÉ
print("\n========================")
print("ANALYSE APPROFONDIE ET MODÈLE AVANCÉ")
print("========================")

# Comparer plus précisément les prédictions des deux modèles
import matplotlib.pyplot as plt
import numpy as np

# 1. Comparer directement les prédictions
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_simple, y_pred_multiple, alpha=0.5)
plt.plot([min(y_pred_simple), max(y_pred_simple)], 
         [min(y_pred_simple), max(y_pred_simple)], 'r--')
plt.xlabel('Prédictions du modèle simple')
plt.ylabel('Prédictions du modèle multiple')
plt.title('Comparaison des prédictions des deux modèles')
plt.grid(True)
plt.savefig('../results/prediction_comparison.png')
plt.show()

# 2. Analyser les coefficients du modèle multiple
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': model_multiple.coef_
}).sort_values('Coefficient', ascending=False)
print("\nCoefficients du modèle multiple:")
print(coef_df)

# 3. Matrice de corrélation entre les features
corr_matrix = X_train.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matrice de corrélation des features')
plt.tight_layout()
plt.savefig('../results/correlation_matrix.png')
plt.show()

# 4. Essayer un modèle plus avancé: Random Forest
from sklearn.ensemble import RandomForestRegressor

# Entraîner un modèle Random Forest
print("\nEntraînement d'un modèle Random Forest...")
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Prédictions
y_pred_rf = model_rf.predict(X_test)

# Métriques
rf_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
rf_r2 = r2_score(y_test, y_pred_rf)

print(f"\nRandom Forest - RMSE: {rf_rmse:.2f}, R²: {rf_r2:.2f}")

# Comparer avec les modèles précédents
print("\nComparaison des trois modèles:")
comparison_updated = pd.DataFrame({
    'Métrique': ['RMSE', 'R²'],
    f'Régression Simple ({best_feature})': [metrics_simple['test_rmse'], metrics_simple['test_r2']],
    'Régression Multiple': [metrics_multiple['test_rmse'], metrics_multiple['test_r2']],
    'Random Forest': [rf_rmse, rf_r2]
})
print(comparison_updated)

# Sauvegarder le modèle Random Forest
import joblib
os.makedirs("../models", exist_ok=True)
joblib.dump(model_rf, "../models/random_forest.pkl")
print("\nModèle Random Forest sauvegardé dans '../models/random_forest.pkl'")

# Visualiser les prédictions du Random Forest
visualize_predictions(y_test, y_pred_rf, "Random Forest")
plt.show()

print("\nModélisation terminée et modèles sauvegardés dans le dossier 'models/'")